In [3]:
import pandas as pd
from datetime import datetime, date
import numpy as np
import os

In [ ]:
import pandas as pd
from datetime import datetime, date
import numpy as np
import os

# Limito la lectura del Padron utilizando solo las columnas necesarias
cols = ['N°', 'NOMBRE', 'Fecha apertura', 'ORGANIZACIÓN ', 'DIRECTOR EXPLOTACIÓN', 'DIRECTOR OPERACIONAL', 'DIRECTOR / GERENTE REGIONAL', 'DIRECTOR/ GERENTE TIENDA', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'DOMICILIO', 'PROVINCIA', 'CAJAS', 'M² Parcking', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']

# Leo el archivo (Conteplar que el archivo debe estar dentro de una carpeta llamada data)
padron = pd.read_excel('data/test/padron.xlsx', header=17, usecols=cols)

# Normalizo los nombres de las columnas
padron.columns = padron.columns.str.lower().str.replace('/', '').str.replace('.2', '').str.strip().str.replace('  ', ' ').str.replace(' ', '_').str.replace('m²', 'mtr')
padron = padron.rename(columns={'n°':'numero_operacional'})

# Elimino las filas que tengan na en su numero, nombre, organizacion
padron.dropna(subset=['numero_operacional', 'nombre', 'organización'], how='any')

# Obtengo unicamente los valores que tengan Numero Operacional y no tengan nulos
padron = padron[(padron['numero_operacional'] != '-') & (padron['numero_operacional'].notna())]

# Convierto la Columna a INT
padron['numero_operacional'] = pd.to_numeric(padron['numero_operacional'], errors='coerce').astype('Int64')

# Me quedo con aquellas lineas que tengan numero Operacional
padron = padron[padron['numero_operacional'].notna()]

# Replazo valores nullos por dummies
padron['fecha_apertura'] = padron['fecha_apertura'].fillna(date(2020, 1, 1))

# Realizo una seleccion de columnas
columnas_categoricas = ['director_explotación', 'director_operacional', 'director_gerente_regional', 'director_gerente_tienda', 'domicilio', 'provincia', 'ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']
date_columns = ['fecha_apertura', 'fin_de_cierre']
num_columns = ['mtr_salón', 'mtr_pgc', 'mtr_pft', 'mtr_bazar', 'mtr_electro', 'mtr_textil', 'mtr_pls', 'mtr_galerias', 'cajas', 'mtr_parcking']
string_columns = ['nombre', 'organización', 'director_explotación', 'director_operacional', 'director_gerente_regional', 'director_gerente_tienda', 'domicilio', 'provincia', 'ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']

# Normalizo algunos valores de las columnas Datetime
padron['fin_de_cierre'] = padron['fin_de_cierre'].replace('-', '')
padron['fin_de_cierre'] = padron['fin_de_cierre'].fillna('')

# Itero sobre las columnas seleccionadas anteriormente para completar, limpiar y normalizar valores. Esto evita errores cuando se convierten a su tipo de datos indicado
for col in columnas_categoricas:
    padron[col] = padron[col].fillna('S/D')

for col in num_columns:
    padron[col] = padron[col].fillna(0)
    padron[col] = padron[col].replace('sd', 0).replace('SD', 0).replace('-', 0)
    padron[col] = padron[col].astype(int)

for col in string_columns:
    padron[col] = padron[col].str.lower().astype(str)

for col in date_columns:
    padron[col] = pd.to_datetime(padron[col])

# Genero una nueva columna condicional que identifica cuales son las tiendas cerradas en base a su fecha de cierre, cuales estan activas en base a a la falta de una fecha de cierre, y aquellas que estan por abrir, comparando su fecha de apertura con la fecha en la cual corre el script
padron['estado_tienda'] = np.where(padron['fecha_apertura'].dt.date > date(datetime.today().year, datetime.today().month, datetime.today().day), 'proxima apertura', np.where(padron['fin_de_cierre'].notna(), 'cerrada', 'activa'))

# Finalmente renombro columnas para que no lleven tilde
padron = padron.rename(columns={
    'organización':'organizacion',
    'director_explotación':'director_explotacion',
    'mtr_salón':'mtr_salon'
})

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_14696\3522769908.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  padron[col] = padron[col].replace('sd', 0).replace('SD', 0).replace('-', 0)
C:\Users\juan_mera\AppData\Local\Temp\ipykernel_14696\3522769908.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  padron[col] = padron[col].replace('sd', 0).replace('SD', 0).replace('-', 0)
C:\Users\juan_mera\AppData\Local\Temp\ipykernel_14696\3522769908.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be r

In [26]:
from pathlib import Path
import os
import gspread
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from gspread_dataframe import get_as_dataframe

# --- Identificadores del archivo ---
sheet_id = "1z8udBUqUe790LT08V3DJHgldnpqwlz2F"
sheet_gid = 123780391

# --- Detectar ruta de credenciales según SO ---
if os.name == 'nt':  # Windows
    cred_path = Path(os.getenv("APPDATA")) / "gcloud" / "application_default_credentials.json"
else:  # Linux / Mac
    cred_path = Path.home() / ".config" / "gcloud" / "application_default_credentials.json"

if not cred_path.exists():
    raise FileNotFoundError(f"No se encontró el archivo de credenciales en: {cred_path}")

print(f"✅ Archivo de credenciales detectado: {cred_path}")

# --- Cargar credenciales ---
creds = Credentials.from_authorized_user_file(
    str(cred_path),
    scopes=[
        "https://www.googleapis.com/auth/spreadsheets.readonly",
        "https://www.googleapis.com/auth/drive.readonly"
    ]
)

print(creds._client_id)
print(creds._quota_project_id)


# Refrescar si está vencido
if creds.expired and creds.refresh_token:
    creds.refresh(Request())

# --- Conectarse a Google Sheets ---
gc = gspread.authorize(creds)
sh = gc.open_by_key(sheet_id)

# Buscar la hoja por GID
worksheet = next((ws for ws in sh.worksheets() if ws.id == sheet_gid), sh.get_worksheet(0))

# --- Leer datos a DataFrame ---
padron = get_as_dataframe(worksheet, evaluate_formulas=True, dtype=str).dropna(how="all")

print("✅ Archivo 'Padrón 2025 Nueva Estructura.xlsx' leído correctamente desde Google Sheets.")
print(f"Total de filas: {len(padron)}")
print(padron.head(10))

✅ Archivo de credenciales detectado: C:\Users\juan_mera\AppData\Roaming\gcloud\application_default_credentials.json
764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com
gcp-ar-cdg-datos-dev


APIError: APIError: [400]: This operation is not supported for this document

In [ ]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import pickle

# SCOPES necesarios para trabajar con Drive y Sheets
SCOPES = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets"
]

# Cargar las credenciales (usando token de OAuth)
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

# Si no hay token o está vencido, iniciar flujo OAuth
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

# Crear cliente de Google Drive
drive_service = build('drive', 'v3', credentials=creds)

# ID del archivo original (el Excel "sagrado")
original_file_id = "1z8udBUqUe790LT08V3DJHgldnpqwlz2F"

# ID de la carpeta de pruebas en tu Drive
folder_id = "1hkiFuMgJIxaMCL5oFfd4fjXYTWB7Jc3b"

# Nombre del nuevo archivo
copied_file_name = "Padrón 2025 (Prueba AutoConvertida)"

# Crear una copia CONVERSIÓN automática a Google Sheets
copied_file = drive_service.files().copy(
    fileId=original_file_id,
    body={
        'name': copied_file_name,
        'parents': [folder_id],
        'mimeType': 'application/vnd.google-apps.spreadsheet'
    }
).execute()

new_file_id = copied_file['id']

print(f"✅ Archivo copiado y convertido: https://docs.google.com/spreadsheets/d/{new_file_id}/edit")